In [1]:
# import packages
# install Google Tesseract OCR and change the tesseract_cmd variable
# see more: https://pypi.org/project/pytesseract/

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract as tess
import glob
import os
import requests
import json
import re

# If you don't have tesseract executable in your PATH, include the following:
# pytesseract.pytesseract.tesseract_cmd = r'<full_path_to_your_tesseract_executable>'

tess.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [2]:
def get_all_filesname(path):
    files = os.listdir(path)
    return files

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [3]:
video_filelist = get_all_filesname("D:\\Video Retrieval System\\keyframes")[:10]
# for full db
#video_filelist = get_all_filesname("D:\\Video Retrieval System\\keyframes")

In [4]:
for videonr in video_filelist:
    f = open(f"D:\\Video Retrieval System\\info\\{videonr}.json")
    data = json.load(f)
    video_url = "http://127.0.0.1:8000/api/updateVideo"
    csrftoken = "csrftoken=qFikZvAWvofmMZA9RkpuOuACmQqExtcM7RhGDrskdWAtNd5lV79Wr0Eh9p2e1jNS"
    video_payload = json.dumps({
        "video_id": int(videonr),
        "description": cleanhtml(data["description"]),
        "title": data["title"],
        "vimeo_id": data["vimeoId"]
        
    })
    headers = {
      'Content-Type': 'application/json',
      'Cookie': csrftoken
    }

    response = requests.request("POST", video_url, headers=headers, data=video_payload)
    
    for tag in data["tags"]:
        tag_url = "http://127.0.0.1:8000/api/updateTags"
        tag_payload = json.dumps({
            "video_id": int(videonr),
            "tag": tag
        })
        response = requests.request("POST", tag_url, headers=headers, data=tag_payload)

    for filename in glob.glob(f'D:\Video Retrieval System\\keyframes\\{videonr}\\*.png'):
        if filename[-16:-13] == "000":
            keyframe_id = filename[-16:-4]
        else:
            keyframe_id = filename[-15:-4]
        img = Image.open(filename)
        text = tess.image_to_string(img).strip("\n\x0c")
        if text != (" " or "") and len(text) > 0:
            text_url = "http://127.0.0.1:8000/api/updateTextanalysis"
            text.replace("/n", " ")

            text_payload = json.dumps({
              "video_id": int(videonr),
              "keyframe_id": "id1235",
              "text": text
            })

            response = requests.request("POST", text_url, headers=headers, data=text_payload)